In [1]:
import json
import pandas as pd
import plotly.graph_objects as go
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import plotly.io as pio

In [33]:
#df = pd.read_excel('data.xlsx')# read data from the file
df_used=pd.read_excel('data_used.xlsx')#read the province list
df_used_name= pd.DataFrame(df_used, columns=["ProvinceName1","ProvinceName2","ProvinceName3","ProvinceName4"])
df_used_EnglishName=pd.DataFrame(df_used, columns=["ProvinceEnglishName1","ProvinceEnglishName2","ProvinceEnglishName3","ProvinceEnglishName4"])
df_used_number=pd.DataFrame(df_used, columns=["Data1","Data2","Data3","Data4"])
with open('china.json', encoding='utf-8') as response:#load the json file
    geojson = json.load(response)
    
def getFigure(range1=None):
    if range1 is None:
        zmin = 0# color show from 0
        zmax = df_now['number'].max() # zmax is the most biggest number in 4 provinces
        colors = trunc_cmap# what range of colour to use 
    else:
        zmin = 0
        zmax = df_now['number'].max()*2 # double range of color
        colors = full_cmap

    fig = go.Figure( 
        go.Choropleth(# use the go library of plotly. using a graph object.  Draw the figure
        locations=df_now['province'],# Spatial coordinates
        z=df_now['number'],#Data to be color-coded
        zmin=zmin,#Sets the lower bound of the color domain. 
        zmax=zmax,#Sets the upper bound of the color domain. 
        geojson=geojson,# layout json
        featureidkey='properties.name',#use this key in the json file to corredpond to the name in 'locations'
        colorscale=colors, # use the color we set before
        #locationmode='geojson-id',# set of locations match entries in `locations`
        colorbar_len=1,# the length of colorbar
        colorbar_y=-0.5,# the location of colorbar is under the graph
        #colorbar=dict(
        #tickvals = [0, 2000, 4000, 6000, 8000,10000,12000,14000],
        #ticks='outside'
        #)
        
    ))

    fig.update_geos(fitbounds="locations",)#Determines if this subplot's view settings are auto-computed to fit trace data
    fig.update_geos(showframe=False,# Set the display details, and do not display the parts irrelevant to the experiment
                          showcoastlines=False,
                          showcountries=False,
                          showland=False,
                          showlakes=False,
    )
    
    fig.update_traces(colorbar_orientation='h',  selector=dict(type='choropleth'))#colorbar is horiziontial
    fig.update_traces(marker_line_color='black', selector=dict(type='choropleth'))# board line's colour
    
    #fig.update_layout(margin=dict(r=20,t=20,b=20,l=20),)

    return fig



for i in range(42):   
    df_now= pd.concat( [pd.DataFrame(df_used_name.values[i]), pd.DataFrame(df_used_EnglishName.values[i]), pd.DataFrame(df_used_number.values[i])], axis=1 )
    df_now.columns = ['province', 'provinceEnglishName','number']# set the column
    print(df_now) # Store the data required for each drawing

    splits = 1000# number of individual colours to use in full version 
    halfway = int(splits/2)# number of individual colours to use in truncated version

    if(i%6==0): #Cycle through colors
        colour_values = mpl.cm.Reds(np.linspace(0, 1, splits))# list of rgb colours for full version. Returns num evenly spaced samples, calculated over the interval [start, stop]
    elif(i%6==1):
        colour_values = mpl.cm.Greys(np.linspace(0, 1, splits))
    elif(i%6==2):
        colour_values = mpl.cm.Purples(np.linspace(0, 1, splits))
    elif(i%6==3):
        colour_values = mpl.cm.Blues(np.linspace(0, 1, splits))
    elif(i%6==4):
        colour_values = mpl.cm.Greens(np.linspace(0, 1, splits))
    elif(i%6==5):
        colour_values = mpl.cm.Oranges(np.linspace(0, 1, splits))
        
        
    trunc_colours = colour_values[0:halfway,:] # all rows, all columns
    trunc_index=np.linspace(0,1,halfway)# splite the 0-1 to get the index of color
    #print(trunc_index)
    trunc_cmap=[]
    for index,value in zip(trunc_index,trunc_colours):# use index and value to iterate in trunc_index and trunc_coloues.create the color which correspond to the index
        trunc_cmap.append([index,'rgb({0:d},{1:d},{2:d})'.format(int(value[0]*255),int(value[1]*255),int(value[2]*255))])#Re match color and index. 

    full_colours = colour_values[:,:] # all rows, all columns
    full_index=np.linspace(0,1,splits)
    full_cmap = []
    for index,value in zip(full_index,full_colours):# full colour range 
        full_cmap.append([index,'rgb({0:d},{1:d},{2:d})'.format(int(value[0]*255),int(value[1]*255),int(value[2]*255))])#Convert the following integers into strings and splice them into RGB strings
    # [0.0, 'rgb(255,245,240)']
    
    figure_name1=str(i+1)+"_narrow.png"# set figure name 
    figure = getFigure()# draw the narrow range color
    pio.write_image(figure, figure_name1)
    #figure.show()

    figure = getFigure(1)# draw the full range color
    figure_name2=str(i+1)+"_full.png"
    pio.write_image(figure, figure_name2)
    #figure.show()


  province provinceEnglishName  number
0     黑龙江省        Heilongjiang      90
1      吉林省               Jilin      90
2      辽宁省            Liaoning     100
3   内蒙古自治区           Neimenggu      80
  province provinceEnglishName  number
0      河北省               Hebei      70
1      吉林省               Jilin      90
2      辽宁省            Liaoning     100
3   内蒙古自治区           Neimenggu      80
  province provinceEnglishName  number
0      河北省               Hebei      70
1      吉林省               Jilin      90
2      辽宁省            Liaoning     100
3      北京市             Beijing     100
  province provinceEnglishName  number
0      河北省               Hebei      70
1      天津市             Tianjin      80
2      辽宁省            Liaoning     100
3      北京市             Beijing     100
  province provinceEnglishName  number
0      河北省               Hebei      70
1      天津市             Tianjin      80
2   内蒙古自治区           Neimenggu      80
3      北京市             Beijing     100
  province provinceEnglis